In [17]:
import time
from collections import namedtuple
from functools import reduce
from queue import SimpleQueue, PriorityQueue
from random import random

import numpy as np

In [18]:
PROBLEM_SIZE = 8
NUM_SETS = 10
SETS = tuple(np.array([random() < .3 for _ in range(PROBLEM_SIZE)]) for _ in range(NUM_SETS))
State = namedtuple('State', ['taken', 'not_taken'])
SETS

(array([False, False, False, False,  True, False, False, False]),
 array([False, False, False, False, False,  True, False,  True]),
 array([False, False,  True, False,  True, False,  True, False]),
 array([False, False, False,  True, False, False, False, False]),
 array([False,  True,  True,  True, False, False,  True, False]),
 array([False, False, False, False, False, False, False, False]),
 array([False,  True,  True, False, False,  True, False, False]),
 array([ True, False, False, False,  True, False,  True, False]),
 array([False, False,  True, False,  True,  True, False, False]),
 array([ True, False,  True,  True, False, False,  True, False]))

In [19]:
def current_cover(state):
    return reduce(np.logical_or, [SETS[i] for i in state.taken], np.array([False for _ in range(PROBLEM_SIZE)]))


def goal_check(state):
    return np.all(current_cover(state))


def distance(state):
    # Multiply by 1.5 to prioritize covering over tree descending
    return (PROBLEM_SIZE - sum(current_cover(state))) * 1.5


def actual_cost(current_state):
    return len(current_state.taken)


def a_star(state, current_state):
    return actual_cost(current_state) + distance(state)

In [20]:
assert goal_check(State(set(range(NUM_SETS)), set())), "Problem not solvable"

In [21]:
frontier = PriorityQueue()
state = State(set(), set(range(NUM_SETS)))
frontier.put((a_star(state, state), state))

start_time = time.time()
counter = 0
_, current_state = frontier.get()
while not goal_check(current_state):
    counter += 1
    for action in current_state.not_taken:
        # Skip empty sets
        if sum(current_cover(State(set() ^ {action}, set(range(NUM_SETS)) ^ {action}))) != 0:
            new_state = State(current_state.taken ^ {action}, current_state.not_taken ^ {action})
            frontier.put((a_star(new_state, current_state), new_state))
    _, current_state = frontier.get()

print(f"Solved in {counter} steps ({len(current_state.taken)} tiles) in {time.time() - start_time}s")

Solved in 3 steps (3 tiles) in 0.0004086494445800781s


In [22]:
print(goal_check(current_state), distance(current_state))

True 0.0


In [23]:
current_state

State(taken={1, 4, 7}, not_taken={0, 2, 3, 5, 6, 8, 9})

In [24]:
[SETS[i] for i in current_state.taken]

[array([False, False, False, False, False,  True, False,  True]),
 array([False,  True,  True,  True, False, False,  True, False]),
 array([ True, False, False, False,  True, False,  True, False])]